In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 为批量预测训练图像分类模型的AutoML

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl_image_classification_batch_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl_image_classification_batch_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/automl_image_classification_online_prediction.ipynb">
       <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex AI SDK创建图像分类模型，并使用Google Cloud的AutoML模型进行批量预测。

了解更多关于[从图像分类模型获取预测](https://cloud.google.com/vertex-ai/docs/image-data/classification/get-predictions)的信息。

### 目标

在本教程中，您将从Python脚本创建一个AutoML图像分类模型，然后使用Vertex SDK进行批量预测。您还可以选择使用`gcloud`命令行工具或在Cloud Console上在线创建和部署模型。

所执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

使用批量预测和在线预测之间存在一个关键区别：

* 预测服务：对整个实例集（即一个或多个数据项）进行按需预测，并实时返回结果。

* 批量预测服务：对整个实例集进行排队（批处理）预测，在后台存储结果，并在准备就绪时将结果存储在Cloud Storage存储桶中。

数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[花卉数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。在本教程中使用的数据集版本存储在公共云存储桶中。训练过的模型可以预测一幅图像属于五种花卉中的哪一种：雏菊、蒲公英、玫瑰、向日葵或郁金香。

### 费用

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage
定价](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预期使用情况生成成本估算。

## 安装

安装 Python 的 Vertex AI SDK 的最新版本。

In [ ]:
import os

! pip3 install --upgrade --quiet google-cloud-aiplatform

if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade --quiet tensorflow

只有 Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在你開始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

1. 顶点 AI 工作台
* 既然您已经通过身份验证，无需再做任何操作。

2. 本地 JupyterLab 实例，取消注释并运行:

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
* 查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上向您的服务账户授予云存储权限。

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在的情况下才运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库和定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform

初始化Python的Vertex AI SDK

为您的项目和对应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

# 教程

现在您已经准备好开始创建您自己的AutoML图像分类模型。

云存储的培训数据位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

迅速查看您的数据

本教程使用存储在公共云存储桶中的鲜花数据集的版本，使用CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中的行数（`wc -l`）来统计示例数量，然后查看前几行数据。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

创建数据集

接下来，使用`ImageDataset`类的`create`方法创建`Dataset`资源，将以下参数传入：

- `display_name`: `Dataset`资源的可读名称。
- `gcs_source`: 一个或多个数据集索引文件列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`: 数据项的数据标记模式。

此操作可能需要几分钟的时间。

In [ ]:
dataset = aiplatform.ImageDataset.create(
    display_name="Flowers",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

### 创建和运行训练流水线

训练 AutoML 模型分为两个步骤：1) 创建一个训练流水线，2) 运行这个流水线。

#### 创建训练流水线

使用 `AutoMLImageTrainingJob` 类创建一个 AutoML 训练流水线，具有以下参数：

- `display_name`：`TrainingJob` 资源的可读名称。
- `prediction_type`：为模型训练指定任务类型。
  - `classification`：图像分类模型。
  - `object_detection`：图像目标检测模型。
- `multi_label`：如果是分类任务，指定是单标签 (`False`) 还是多标签 (`True`)。
- `model_type`：用于部署的模型类型。
  - `CLOUD`：在 Google Cloud 上部署。
  - `CLOUD_HIGH_ACCURACY_1`：优化准确性而非延迟，适用于在 Google Cloud 上部署。
  - `CLOUD_LOW_LATENCY_`：优化延迟而非准确性，适用于在 Google Cloud 上部署。
  - `MOBILE_TF_VERSATILE_1`：在边缘设备上部署。
  - `MOBILE_TF_HIGH_ACCURACY_1`：优化准确性而非延迟，适用于在边缘设备上部署。
  - `MOBILE_TF_LOW_LATENCY_1`：优化延迟而非准确性，适用于在边缘设备上部署。
- `base_model`：（可选）从现有的 `Model` 资源进行迁移学习，仅支持图像分类。

实例化的对象是用于训练任务的有向无环图（DAG）。

In [ ]:
dag = aiplatform.AutoMLImageTrainingJob(
    display_name="flowers",
    prediction_type="classification",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

print(dag)

运行训练流水线

接下来，您可以通过调用方法`run`来运行DAG，从而启动训练任务，参数如下：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集的百分比。
- `test_fraction_split`：用于测试（保留数据）的数据集的百分比。
- `validation_fraction_split`：用于验证的数据集的百分比。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果设置为`True`，则在服务认为无法进一步改善模型目标测量之前，训练可能会在使用整个预算之前完成。

完成`run`方法后会返回`Model`资源。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="flowers",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

## 检查模型评估分数

在模型训练完成后，您可以使用`list_model_evaluations()`方法查看其评估分数。此方法将返回每个评估切片的迭代器。

发送批量预测请求

向您部署的模型发送批量预测。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

获取测试项目

现在对您的Vertex模型进行批量预测。您将使用数据集中的任意示例作为测试项目。不用担心这些示例很可能被用于训练模型 — 我们只是想演示如何进行预测。

In [ ]:
test_items = !gsutil cat $IMPORT_FILE | head -n2
if len(str(test_items[0]).split(",")) == 3:
    _, test_item_1, test_label_1 = str(test_items[0]).split(",")
    _, test_item_2, test_label_2 = str(test_items[1]).split(",")
else:
    test_item_1, test_label_1 = str(test_items[0]).split(",")
    test_item_2, test_label_2 = str(test_items[1]).split(",")

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### 复制测试项目

对于批量预测，请将测试项目复制到您的云存储桶中。

In [ ]:
file_1 = test_item_1.split("/")[-1]
file_2 = test_item_2.split("/")[-1]

! gsutil cp $test_item_1 $BUCKET_URI/$file_1
! gsutil cp $test_item_2 $BUCKET_URI/$file_2

test_item_1 = BUCKET_URI + "/" + file_1
test_item_2 = BUCKET_URI + "/" + file_2

### 创建批量输入文件

现在创建一个批量输入文件，将其存储在您的本地云存储桶中。批量输入文件可以是CSV或JSONL格式。在本教程中将使用JSONL格式。对于JSONL文件，您需要为每个数据项（实例）的每一行创建一个字典条目。字典包含键/值对：

- `content`：图像的云存储路径。
- `mime_type`：内容类型。在我们的示例中，它是一个`jpeg`文件。

例如：

                        {'content'：'[您的存储桶]/file1.jpg'，'mime_type'：'jpeg'}

In [ ]:
import json

import tensorflow as tf

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": test_item_1, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")
    data = {"content": test_item_2, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

进行批量预测请求

现在您的模型资源已经训练完成，您可以通过调用batch_predict()方法进行批量预测，需要以下参数：

- 'job_display_name': 批量预测作业的可读名称。
- 'gcs_source': 一个或多个批量请求输入文件的列表。
- 'gcs_destination_prefix': 用于存储批量预测结果的Cloud Storage位置。
- 'sync': 如果设置为True，则调用将会阻塞，等待异步批处理作业完成。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="flowers",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

### 等待批处理预测任务完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测作业完成。

In [ ]:
batch_predict_job.wait()

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果被写入了您在批量预测请求中指定的云存储输出存储桶中。您可以调用`iter_outputs()`方法来获取生成的每个带有结果的云存储文件的列表。每个文件以JSON格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。
 - `ids`：每个预测请求的内部分配的唯一标识符。
 - `displayNames`：每个类标签的类名。
 - `confidences`：每个类标签的预测置信度，介于0和1之间。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

清理

要清理本项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = False

# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML trainig job
dag.delete()

# Delete the batch prediction job
batch_predict_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI